# Data Prep

In [ ]:
!pip install transformers datasets optuna tensorboard
!pip install accelerate -U
!pip install transformers -U

In [ ]:
import pandas as pd

sum_df = pd.read_csv('/content/drive/MyDrive/TrainingBart/summary_dataset.csv')
sum_df.head()
ori_df = pd.read_csv('/content/drive/MyDrive/TrainingBart/earning_calls_transcript_1k.csv')
ori_df.head()

ori_df = ori_df.head(1000)
ori_df.index = sum_df.index
sum_df['EarningCallTranscript'] = ori_df['EarningCallTranscript']
sum_df.info()


sum_df.to_csv("final_dataset.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Exchange               1000 non-null   object
 1   Symbol                 1000 non-null   object
 2   Year                   1000 non-null   int64 
 3   EarningCallTranscript  1000 non-null   object
 4   Summary                1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


# Training

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.integrations import TensorBoardCallback
import tensorflow as tf
import os

# Verify installations
import accelerate
import transformers

In [ ]:
df = sum_df
dataset = Dataset.from_pandas(df)
dataset = dataset.rename_column("Summary", "target")
dataset = dataset.rename_column("EarningCallTranscript", "source")

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

In [ ]:
def chunk_text(text, chunk_size=1024):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def tokenize_function(examples):
    input_ids = []
    attention_mask = []
    labels = []

    for source_text, target_text in zip(examples['source'], examples['target']):
        source_text = ' '.join(source_text) if isinstance(source_text, list) else source_text
        target_text = ' '.join(target_text) if isinstance(target_text, list) else target_text

        source_chunks = chunk_text(source_text, chunk_size=1024)
        target_chunks = chunk_text(target_text, chunk_size=1024)

        for source_chunk, target_chunk in zip(source_chunks, target_chunks):
            model_inputs = tokenizer(source_chunk, max_length=1024, truncation=True, padding="max_length")
            labels_chunk = tokenizer(target_chunk, max_length=1024, truncation=True, padding="max_length")

            input_ids.append(model_inputs['input_ids'])
            attention_mask.append(model_inputs['attention_mask'])
            labels.append(labels_chunk['input_ids'])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

# Tokenize dataset
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,  # Process in batches for efficiency
    batch_size=8,  # Adjust batch size based on memory constraints
    remove_columns=dataset.column_names,  # Ensure columns are correctly replaced
    num_proc=4
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [ ]:
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2.8023652601511377e-05,  # Best hyperparameter for learning rate
    per_device_train_batch_size=2,  # Best hyperparameter for train batch size
    per_device_eval_batch_size=3,  # Best hyperparameter for eval batch size
    weight_decay=0.08233597332385283,  # Best hyperparameter for weight decay
    num_train_epochs=25,  # Best hyperparameter for number of epochs
    gradient_accumulation_steps=1,  # Best hyperparameter for gradient accumulation steps
    save_total_limit=3,
    save_steps=500,  # Save model every 500 steps
    predict_with_generate=True,
    fp16=True,  # Enables mixed precision training
    logging_dir=log_dir,
    logging_steps=10,  # Adjust logging frequency as needed
    report_to=["tensorboard"]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[TensorBoardCallback()]
)


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

TensorFlow version: 2.15.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.897800,1.551371
2,1.516900,1.505263
3,1.446000,1.486049
4,1.270700,1.479183
5,1.066200,1.477510
6,1.245000,1.486988
7,1.095100,1.498129
8,1.189200,1.516799
9,0.960500,1.531908
10,1.146100,1.550574


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

Epoch,Training Loss,Validation Loss
1,1.897800,1.551371
2,1.516900,1.505263
3,1.446000,1.486049
4,1.270700,1.479183
5,1.066200,1.477510
6,1.245000,1.486988
7,1.095100,1.498129
8,1.189200,1.516799
9,0.960500,1.531908
10,1.146100,1.550574


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=11250, training_loss=1.0065902219136555, metrics={'train_runtime': 5023.2081, 'train_samples_per_second': 4.479, 'train_steps_per_second': 2.24, 'total_flos': 4.875985354752e+16, 'train_loss': 1.0065902219136555, 'epoch': 25.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/TrainingBart/FineTuned")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


#Testing


In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
from transformers import BartTokenizer, BartForConditionalGeneration
from bert_score import score

# Load the fine-tuned model and tokenizer
model_dir = "/content/drive/MyDrive/TrainingBart/FineTuned"
tokenizer = BartTokenizer.from_pretrained(model_dir)
model = BartForConditionalGeneration.from_pretrained(model_dir)

# Load your original DataFrame
df = pd.read_csv("/content/drive/MyDrive/TrainingBart/earning_calls_transcript_1k.csv")

# Function to chunk text
def chunk_text(text, chunk_size=1024):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Function to summarize a single chunk
def summarize_chunk(chunk):
    inputs = tokenizer.encode("summarize: " + chunk, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Sample 100 random transcripts
sampled_df = df.sample(n=100, random_state=42)

# Lists to store results
original_transcripts = []
summaries = []
precision_scores = []
recall_scores = []
f1_scores = []

# Process each transcript
for transcript in sampled_df["EarningCallTranscript"]:
    # Chunk the transcript
    chunks = chunk_text(transcript, chunk_size=1024)
    chunk_summaries = [summarize_chunk(chunk) for chunk in chunks]

    # Combine chunk summaries
    combined_summary = ' '.join(chunk_summaries)

    # Compute BERTScore
    P, R, F1 = score([combined_summary], [transcript], lang="en", verbose=True)

    # Store the results
    original_transcripts.append(transcript)
    summaries.append(combined_summary)
    precision_scores.append(P.mean().item())
    recall_scores.append(R.mean().item())
    f1_scores.append(F1.mean().item())

# Create a new DataFrame with the results
results_df = pd.DataFrame({
    'OriginalTranscript': original_transcripts,
    'Summary': summaries,
    'BERTScore_Precision': precision_scores,
    'BERTScore_Recall': recall_scores,
    'BERTScore_F1': f1_scores
})

# Save the results to a CSV file
results_df.to_csv("random_100_earning_call_summaries_with_bertscore.csv", index=False)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.27 seconds, 3.70 sentences/sec
